Uploader le fichier products.csv. Repérer les valeurs aberrantes dans le fichier (ordre de grandeur : plusieurs centaines).

In [0]:
from pyspark.sql.functions import col

In [0]:
df_products = spark.read.csv("/FileStore/tables/SPN_products.csv", header=True, inferSchema=True)

In [0]:
df_products.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- code: double (nullable = true)
 |-- fat_100g: double (nullable = true)
 |-- saturated-fat_100g: double (nullable = true)
 |-- sugars_100g: double (nullable = true)
 |-- fiber_100g: double (nullable = true)
 |-- proteins_100g: double (nullable = true)
 |-- salt_100g: double (nullable = true)
 |-- sodium_100g: double (nullable = true)
 |-- autre: double (nullable = true)



In [0]:
display(df_products.describe())

summary,_c0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
count,320772,320749,243891,229554,244971,200886,259922,255510,255463,320772
mean,160385.5,3.8241411905082087E33,12.730378571710334,5.129932302769851,16.003483547358233,2.8621109473035657,7.075939659205786,2.0286239493368763,0.7988154570102396,65.86151103976306
stddev,92599.04461170212,1.8551307088505311E36,17.578746693167723,8.01423813836071,22.327284404375717,12.86757803197955,8.409054244714131,128.26945441693383,50.50442790314077,32.09102128093654
min,0,1.0,0.0,0.0,-17.86,-6.7,-800.0,0.0,0.0,0.0
max,320771,1.0325807005530831E39,714.29,550.0,3520.0,5380.0,430.0,64312.8,25320.0,889.38


In [0]:
df_products.show()

+---+-------+--------+------------------+-----------+----------+-------------+---------+-----------+------------------+
|_c0|   code|fat_100g|saturated-fat_100g|sugars_100g|fiber_100g|proteins_100g|salt_100g|sodium_100g|             autre|
+---+-------+--------+------------------+-----------+----------+-------------+---------+-----------+------------------+
|  0| 3087.0|    null|              null|       null|      null|         null|     null|       null|             100.0|
|  1| 4530.0|   28.57|             28.57|      14.29|       3.6|         3.57|      0.0|        0.0|21.400000000000006|
|  2| 4559.0|   17.86|               0.0|      17.86|       7.1|        17.86|    0.635|       0.25|            38.435|
|  3|16087.0|   57.14|              5.36|       3.57|       7.1|        17.86|  1.22428|      0.482| 7.263720000000021|
|  4|16094.0|    1.43|              null|       null|       5.7|         8.57|     null|       null|              84.3|
|  5|16100.0|   18.27|              1.92

In [0]:
filter_condition = None
columns_to_check = [
    "fat_100g",
    "saturated-fat_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    "sodium_100g",
    "autre",
]
for column in columns_to_check:
    condition = col(column) < 0
    filter_condition = (filter_condition | condition)

df_negative = df_products.filter(filter_condition)

In [0]:
df_negative.count()

Out[18]: 11

In [0]:
df_sum_more_than_100 = df_products.filter(
    (
        col("fat_100g") +
        col("saturated-fat_100g") +
        ....
    ) > 102
    )
)

  File <command-214145427641576>:6
    ) > 102
    ^
SyntaxError: invalid syntax


In [0]:
columns_to_sum = [
    "fat_100g",
    #"saturated-fat_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    #"sodium_100g",
    "autre",
]
sum_expression = sum(col(column) for column in columns_to_sum)
df_sum_more_than_100 = df_products.filter(sum_expression > 102)
df_sum_more_than_100.count()

Out[22]: 9642

In [0]:
df_sf_more_than_fat = df_products.filter(
    col("saturated-fat_100g") > col("fat_100g")
)


df_1 = df_products.fillna(0)
df_2 = df_1.filter(~((col("salt_100g") >= 2.5 * col("sodium_100g")) & (col("salt_100g") <= 2.58 * col("sodium_100g"))))

df_2.show()

+------+-----------------+--------+------------------+-----------+----------+-------------+---------+-----------+----------+
|   _c0|             code|fat_100g|saturated-fat_100g|sugars_100g|fiber_100g|proteins_100g|salt_100g|sodium_100g|     autre|
+------+-----------------+--------+------------------+-----------+----------+-------------+---------+-----------+----------+
|192840|3.083680994613E12|     0.0|               0.0|        0.0|       0.0|          0.0|   0.1016|        0.0|   99.8984|
|269269|5.000118061905E12|     0.0|               0.0|        0.0|       0.0|          0.0|0.0100076|        0.0|99.9899924|
|275086|5.098732000752E12|     0.3|               0.2|       47.0|       0.0|          0.1| 0.100076|        0.0| 52.299924|
|282383|5.906827001303E12|     3.0|               0.6|        0.7|       6.0|         13.0|   0.0254|        0.0|   76.6746|
+------+-----------------+--------+------------------+-----------+----------+-------------+---------+-----------+----------+
